In [1]:
import jax
import jax.numpy as jnp

In [2]:
from nebula.cases.airfoil import CamberThicknessAirfoil
from nebula.render.tesselation import Tesselator
from nebula.topology import Faces, Shells, Solids, Wires

def make_mesh(
    hub_inlet_angle: jnp.ndarray,
    hub_outlet_angle: jnp.ndarray,
    chord_length: jnp.ndarray,
    mean_inlet_angle: jnp.ndarray,
    mean_outlet_angle: jnp.ndarray,
):
    thickness_ratios = jnp.array([0.015, 0.05, 0.05, 0.015])

    tip_inlet_angle = jnp.radians(-68.773)
    tip_outlet_angle = jnp.radians(16.299)
    hub_rotor_airfoil = CamberThicknessAirfoil(
        inlet_angle=hub_inlet_angle,
        outlet_angle=hub_outlet_angle,
        upper_thick_prop=thickness_ratios,
        lower_thick_prop=thickness_ratios,
        leading_prop=jnp.array(0.5),
        trailing_prop=jnp.array(0.5),
        chord_length=chord_length,
        leading_ctrl_pnt=jnp.array([0.0, 0.0, 0.054]),
    )
    hub_top, hub_bottom = hub_rotor_airfoil.get_edges()

    mean_rotor_airfoil = CamberThicknessAirfoil(
        inlet_angle=mean_inlet_angle,
        outlet_angle=mean_outlet_angle,
        upper_thick_prop=thickness_ratios,
        lower_thick_prop=thickness_ratios,
        leading_prop=jnp.array(0.5),
        trailing_prop=jnp.array(0.5),
        chord_length=chord_length,
        leading_ctrl_pnt=jnp.array([0.0, 0.0, 0.083]),
    )
    mean_top, mean_bottom = mean_rotor_airfoil.get_edges()

    tip_rotor_airfoil = CamberThicknessAirfoil(
        inlet_angle=tip_inlet_angle,
        outlet_angle=tip_outlet_angle,
        upper_thick_prop=thickness_ratios,
        lower_thick_prop=thickness_ratios,
        leading_prop=jnp.array(0.5),
        trailing_prop=jnp.array(0.5),
        chord_length=chord_length,
        leading_ctrl_pnt=jnp.array([0.0, 0.0, 0.111]),
    )
    tip_top, tip_bottom = tip_rotor_airfoil.get_edges()

    top_wires = Wires.skin([hub_top, mean_top, tip_top])
    bottom_wires = Wires.skin([hub_bottom, mean_bottom, tip_bottom])
    combined_wires: Wires = Wires.combine([top_wires, bottom_wires])

    faces = Faces.from_wires(combined_wires, is_exterior=True)
    shells = Shells.from_faces(faces)
    solids = Solids.from_shells(shells)



    return Tesselator.get_differentiable_mesh(solids)

In [7]:
hub_inlet_angle = jnp.radians(-52.261)
hub_outlet_angle = jnp.radians(-57.951)
chord_length = jnp.array(0.018)
mean_inlet_angle = jnp.radians(-57.067)
mean_outlet_angle = jnp.radians(-24.959)

In [20]:
mesh_jacobian = jax.jacobian(
    lambda *args, **kwargs: make_mesh(*args, **kwargs).vertices,
    (2,)
)(hub_inlet_angle, hub_outlet_angle, chord_length, mean_inlet_angle, mean_outlet_angle)

jac_mag_wrt_chord_length = jnp.linalg.norm(mesh_jacobian[0],  axis=-1)
mesh = make_mesh(
    hub_inlet_angle,
    hub_outlet_angle,
    chord_length,
    mean_inlet_angle,
    mean_outlet_angle,
)

In [25]:
from nebula.render.visualization import show

show(mesh, "plot", jac_mag_wrt_chord_length, name="Axial Compressor Rotor Blade CAD Sensitivity w.r.t chord length<br><sup>Nebula by Open Orion</sup>")